In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 29 11:43:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD 



#embedding
from gensim.models import Word2Vec

# tqdm
from tqdm.notebook import tqdm



In [5]:
train_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/train.json"
test_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/test.json"

with open(train_file_path, 'r') as f:
    data = json.load(f)
df_train = pd.DataFrame(data)

with open(test_file_path, 'r') as f:
    data = json.load(f)
df_test = pd.DataFrame(data)

In [6]:
# Retrive the non-prolific author and prolific author index
non_prolific_author_index = []
prolific_author_index = []
for i,j in enumerate(df_train['authors']):
  sum = 0
  for k in j:
    if k < 100 and k >= 0:
      sum += 1
  if sum == 0:
    non_prolific_author_index.append(i)
  else:
    prolific_author_index.append(i)

df_non_prolifc_authors = df_train.drop(prolific_author_index)
df_prolific_authors = df_train.drop(non_prolific_author_index)

In [7]:
df_prolific_authors

,authors,year,abstract,venue,title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [8]:
# Retrieve the prolific author as label
prolific_author_label = []
non_prolific_author_label = []
for i in df_prolific_authors['authors']:
  prolific_author_label_each_instance = []
  non_prolific_author_label_each_instance = []
  for j in i:
    if j >= 0 and j < 100:
      prolific_author_label_each_instance.append(j)
    else:
      non_prolific_author_label_each_instance.append(j)
  prolific_author_label.append(prolific_author_label_each_instance)
  non_prolific_author_label.append(non_prolific_author_label_each_instance)


df_prolific_authors['coauthors'] = non_prolific_author_label

In [9]:
#One-hot encoding the labels
multilable_binarizer = MultiLabelBinarizer()
ohe_prolific_labels = multilable_binarizer.fit_transform(prolific_author_label).astype('float')

In [10]:
# df_prolific_authors['sentences'] = df_prolific_authors['title'] + df_prolific_authors['abstract'] 
# df_prolific_authors = df_prolific_authors.drop(['abstract','title'],axis = 1)

# df_test['sentences'] = df_test['title'] + df_test['abstract'] 
# df_test = df_test.drop(['abstract','title'],axis = 1)

In [11]:
df_prolific_authors

,authors,year,abstract,venue,title,coauthors
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...",[13720]
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[1359, 15881]"
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...",[]
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...",[19617]
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160...","[9641, 5623]"
...,...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359...","[19974, 8903]"
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3...","[6353, 3037, 15856, 13521]"
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1...","[7686, 3810, 7451]"
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152...",[14864]


In [12]:
def convert_to_string(df):
  total_list = []
  for i in df:
    per_instance = []
    for j in i:
      per_instance.append(str(j))
    total_list.append(per_instance)
  return total_list

# word to vec title model
string_title = convert_to_string(df_train['title'])
# Word2Vec
model_title = Word2Vec(string_title, min_count=1)
print(model_title)

# word to vec abstract model
string_abstract = convert_to_string(df_train['abstract'])
# Word2Vec
model_abstract = Word2Vec(string_abstract, min_count=1)
print(model_abstract)


# word to vec coauthor model
string_coauthor = convert_to_string(df_train['authors'])

model_coauthor = Word2Vec(string_coauthor, min_count=1)
print(model_coauthor)








Word2Vec(vocab=3444, size=100, alpha=0.025)
Word2Vec(vocab=4845, size=100, alpha=0.025)
Word2Vec(vocab=20433, size=100, alpha=0.025)


## title and abstract word to embedding

In [13]:
# # title embedding
# prolific_author_string_title = convert_to_string(df_prolific_authors['title'])
# title_embedding = []
# for i in tqdm(prolific_author_string_title):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_title[j]
#   title_embedding.append(temp.mean(axis=0))


# # abstract embedding
# prolific_author_string_abstract = convert_to_string(df_prolific_authors['abstract'])
# abstract_embedding = []
# for i in tqdm(prolific_author_string_abstract):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_abstract[j]
#   abstract_embedding.append(temp.mean(axis=0))





# # coauthor embedding
# prolific_author_string_coauthor = convert_to_string(df_prolific_authors['coauthors'])
# coauthor_embedding = []
# for i in tqdm(prolific_author_string_coauthor):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_coauthor[j]
#   coauthor_embedding.append(temp.mean(axis=0))








## counter vectorizer

In [14]:
# counts for abstract, title and coauthor
def convert_to_instance_string(df):
  total_list = []
  for i in df:
    total_list.append(str(i)[1:-1].replace(",", ""))
  return total_list

# Counts for abstract
abstract_voc = {}
abstract_num = 4999
for i in range(abstract_num):
  abstract_voc[str(i)] = i




prolific_author_string_abstract = convert_to_instance_string(df_prolific_authors['abstract'])
abstract_vectorizer = CountVectorizer(vocabulary = abstract_voc)
abstract_counts = abstract_vectorizer.fit_transform(prolific_author_string_abstract)
abstract_counts = abstract_counts.toarray()



# Counts for title
title_voc = {}
title_num = 4999
for i in range(title_num):
  title_voc[str(i)] = i




prolific_author_string_title = convert_to_instance_string(df_prolific_authors['title'])
title_vectorizer = CountVectorizer(vocabulary = title_voc)
title_counts = title_vectorizer.fit_transform(prolific_author_string_title)
title_counts = title_counts.toarray()


# Counts for coauthor

coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

prolific_author_string_coauthors = convert_to_instance_string(df_prolific_authors['coauthors'])
coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
coauthor_counts = coauthor_vectorizer.fit_transform(prolific_author_string_coauthors)
coauthor_counts = coauthor_counts.toarray()




## Truncated svd for counted-based abstract and title

In [15]:
svd_abstract = TruncatedSVD(n_components=200)
svd_count_abstract = svd_abstract.fit_transform(abstract_counts)

svd_title = TruncatedSVD(n_components=100)
svd_count_title = svd_title.fit_transform(title_counts)

svd_coauthor = TruncatedSVD(n_components=100)
svd_count_coauthor = svd_coauthor.fit_transform(coauthor_counts)


## svd counts abstract and title

In [16]:
# counter vectorized features



coauthors_features_name = ['coauthors_' + str(i) for i in range(100)]
df_coauthors_features = pd.DataFrame(svd_count_coauthor)
df_coauthors_features = df_coauthors_features.astype('float')
df_coauthors_features.columns = coauthors_features_name


abstract_features_name = ['abstract_' + str(i) for i in range(200)]
df_abstract_features = pd.DataFrame(svd_count_abstract)
df_abstract_features = df_abstract_features.astype('float')
df_abstract_features.columns = abstract_features_name

title_features_name = ['title_' + str(i) for i in range(100)]
df_title_features = pd.DataFrame(svd_count_title)
df_title_features = df_title_features.astype('float')
df_title_features.columns = title_features_name


In [17]:
df_merge_coauthors_sentences = pd.concat([df_coauthors_features,df_abstract_features], axis = 1)

In [18]:
df_merge_coauthors_sentences

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,abstract_190,abstract_191,abstract_192,abstract_193,abstract_194,abstract_195,abstract_196,abstract_197,abstract_198,abstract_199
0,1.673357e-05,6.040627e-05,1.963781e-04,-1.253355e-05,-6.574915e-05,4.363344e-05,3.768236e-06,8.071575e-05,6.402486e-05,-1.345032e-04,...,0.315809,0.416685,-0.270793,-0.300086,0.186845,0.191031,0.225105,0.205397,-0.306196,-0.152806
1,1.248832e-06,-1.035409e-05,3.772838e-05,3.516315e-05,1.330434e-04,-4.116083e-05,6.893407e-04,6.493493e-05,5.076847e-05,3.236382e-04,...,-0.175734,-0.356618,0.004239,-0.131291,-0.252497,-0.152170,0.141408,-0.105728,0.643679,-0.076453
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.147875,-0.141530,0.119292,0.415281,-0.182083,-0.033174,0.137858,0.437233,-0.293811,-0.066304
3,3.683431e-05,-6.699412e-05,2.539188e-04,4.683407e-04,3.188861e-05,2.922718e-04,2.541697e-05,-2.492010e-04,-6.449171e-04,-1.908254e-03,...,-0.091786,0.658891,-0.031013,-0.731506,0.524027,0.125519,0.365563,-0.560976,0.263906,0.508082
4,6.796455e-04,-5.289627e-04,8.296649e-05,-1.265077e-04,-1.882584e-04,1.616177e-04,9.587288e-04,-2.064719e-04,3.122119e-03,-5.749697e-04,...,-0.316532,0.544559,0.134363,-0.209656,0.609228,0.904788,0.271621,0.272284,0.548494,-0.602611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,1.886147e-03,-6.705072e-04,7.743749e-03,-1.531516e-04,-6.254286e-04,-2.110889e-03,1.316733e-04,-4.578399e-03,4.243948e-04,2.020365e-02,...,0.369623,-0.025368,-0.089112,-0.140927,0.237865,0.104186,-0.045017,-0.315925,0.342902,0.379778
7456,-6.260789e-09,-8.772704e-08,-6.050563e-08,1.312445e-07,-1.163417e-07,2.362103e-07,3.685576e-07,1.141631e-07,-2.666590e-07,-9.742122e-08,...,-0.428183,-0.691371,0.047509,0.580997,-0.004254,0.392903,-0.198343,0.210394,-0.106948,0.222788
7457,1.017693e-01,5.897996e-02,-4.582297e-02,2.328139e-03,-3.214850e-03,3.109516e-02,3.989235e-04,2.466029e-02,-1.558571e-02,1.797524e-02,...,-0.925530,-0.854843,0.811911,0.294073,-0.262966,-0.409887,-0.882525,0.514422,0.684695,0.056088
7458,3.191306e-04,7.442427e-04,3.109132e-03,-1.893110e-03,-1.009621e-03,-7.476141e-04,1.268684e-04,-1.047950e-03,1.168916e-05,-2.725216e-03,...,0.288163,-0.013864,0.117686,-0.157926,0.268565,-0.252333,-0.186039,-0.290936,0.396492,0.035808


In [19]:
df_coauthors_features

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,coauthors_90,coauthors_91,coauthors_92,coauthors_93,coauthors_94,coauthors_95,coauthors_96,coauthors_97,coauthors_98,coauthors_99
0,1.673357e-05,6.040627e-05,1.963781e-04,-1.253355e-05,-6.574915e-05,4.363344e-05,3.768236e-06,8.071575e-05,6.402486e-05,-1.345032e-04,...,-0.000204,0.000292,-0.000874,-0.000514,-0.000318,0.000274,-0.000131,-0.000394,0.000085,0.001874
1,1.248832e-06,-1.035409e-05,3.772838e-05,3.516315e-05,1.330434e-04,-4.116083e-05,6.893407e-04,6.493493e-05,5.076847e-05,3.236382e-04,...,0.004970,0.021364,-0.018842,-0.012383,-0.003903,0.007565,-0.001069,0.010369,0.033685,-0.005918
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.683431e-05,-6.699412e-05,2.539188e-04,4.683407e-04,3.188861e-05,2.922718e-04,2.541697e-05,-2.492010e-04,-6.449171e-04,-1.908254e-03,...,0.001906,0.001202,0.006184,0.000955,-0.003214,-0.003736,-0.004073,-0.003614,0.000532,0.005745
4,6.796455e-04,-5.289627e-04,8.296649e-05,-1.265077e-04,-1.882584e-04,1.616177e-04,9.587288e-04,-2.064719e-04,3.122119e-03,-5.749697e-04,...,0.020241,-0.000459,0.053145,0.081977,0.048672,0.046181,0.023558,0.069978,0.089546,0.007903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,1.886147e-03,-6.705072e-04,7.743749e-03,-1.531516e-04,-6.254286e-04,-2.110889e-03,1.316733e-04,-4.578399e-03,4.243948e-04,2.020365e-02,...,0.072170,0.023942,0.064311,0.002307,-0.050418,-0.002303,0.031835,-0.016115,0.108072,0.025399
7456,-6.260789e-09,-8.772704e-08,-6.050563e-08,1.312445e-07,-1.163417e-07,2.362103e-07,3.685576e-07,1.141631e-07,-2.666590e-07,-9.742122e-08,...,0.000008,-0.000018,0.000015,-0.000042,0.000050,0.000047,-0.000009,0.000045,-0.000070,0.000026
7457,1.017693e-01,5.897996e-02,-4.582297e-02,2.328139e-03,-3.214850e-03,3.109516e-02,3.989235e-04,2.466029e-02,-1.558571e-02,1.797524e-02,...,0.031759,-0.029596,-0.026854,-0.005487,-0.013196,-0.013511,-0.003997,0.001603,-0.015767,0.039389
7458,3.191306e-04,7.442427e-04,3.109132e-03,-1.893110e-03,-1.009621e-03,-7.476141e-04,1.268684e-04,-1.047950e-03,1.168916e-05,-2.725216e-03,...,-0.134556,0.041365,-0.080506,0.001258,-0.053595,-0.170371,0.170451,-0.245380,0.152230,0.037505


In [20]:
df_title_features

,title_0,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,...,title_90,title_91,title_92,title_93,title_94,title_95,title_96,title_97,title_98,title_99
0,1.295959,0.743809,0.268806,0.046529,0.186041,0.437621,0.143332,-1.221189,0.037354,-0.396264,...,0.273391,-0.023053,0.100232,0.029051,0.100154,-0.020886,-0.037887,0.201098,-0.070926,-0.163205
1,1.700133,-0.912875,-1.319707,-0.042984,1.274011,-0.483756,0.242461,1.136059,0.076590,-0.291597,...,0.069384,-0.155330,0.053081,0.030088,0.025829,-0.080119,0.045167,0.219128,-0.125519,0.099774
2,1.276108,0.053626,-0.310323,1.162363,-1.722691,-0.650924,0.454068,-0.325668,-0.598340,0.287285,...,-0.146725,-0.015530,0.130612,0.192247,-0.325066,0.181985,0.020150,-0.139800,-0.134327,-0.047500
3,0.954030,-1.089008,1.186916,-0.661090,-0.456620,0.272536,0.405763,0.656091,0.428468,-0.324923,...,-0.291318,0.135734,0.462635,-0.409534,0.551033,-0.305430,0.120695,-0.227463,-0.051964,-0.006315
4,2.567098,-1.693997,-0.928658,1.565279,2.520927,-0.496180,0.477821,-0.535965,-0.599013,0.483002,...,0.633095,0.218375,-0.163429,0.333545,0.133231,0.034075,0.094934,-0.306867,-0.138246,-0.030890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,0.909058,0.022548,-0.183789,0.101173,-0.404722,0.316167,0.435812,-0.879759,-0.093336,-0.389909,...,0.080400,0.346376,-0.126127,-0.138508,0.003917,-0.016060,0.087580,-0.166436,-0.015539,0.023294
7456,1.602069,-0.196450,-1.007608,-0.494176,-0.516510,1.359324,0.574036,0.474455,-1.692473,0.360308,...,0.267685,0.132964,0.172446,-0.259745,-0.041689,0.247401,-0.152159,-0.503394,0.130203,-0.174319
7457,0.554212,-0.349605,-0.134213,0.623060,0.661159,0.093371,-0.012643,0.279980,0.014463,0.112096,...,0.097190,0.084577,-0.211680,-0.047919,-0.129362,-0.018182,-0.216502,0.037011,-0.305947,0.145474
7458,1.742442,0.594856,0.100028,0.763121,-1.256499,-0.431276,0.439072,0.286754,-0.393655,0.377043,...,0.197340,-0.383426,-0.051976,0.254383,0.034286,-0.318077,0.022253,0.070260,-0.032666,0.149270


In [21]:
df_abstract_features

,abstract_0,abstract_1,abstract_2,abstract_3,abstract_4,abstract_5,abstract_6,abstract_7,abstract_8,abstract_9,...,abstract_190,abstract_191,abstract_192,abstract_193,abstract_194,abstract_195,abstract_196,abstract_197,abstract_198,abstract_199
0,7.470278,-1.553237,2.482423,2.917728,-2.793129,-0.666076,0.593493,1.051792,3.272780,-0.674373,...,0.315809,0.416685,-0.270793,-0.300086,0.186845,0.191031,0.225105,0.205397,-0.306196,-0.152806
1,9.912461,-4.891517,0.512769,-1.225719,1.987015,-0.756909,0.837022,2.544812,0.221440,1.034592,...,-0.175734,-0.356618,0.004239,-0.131291,-0.252497,-0.152170,0.141408,-0.105728,0.643679,-0.076453
2,8.742746,2.215522,-1.240042,2.013834,-1.437530,-1.259191,-0.204400,1.348229,-0.564419,1.275207,...,0.147875,-0.141530,0.119292,0.415281,-0.182083,-0.033174,0.137858,0.437233,-0.293811,-0.066304
3,9.543987,-1.261974,2.289460,1.961829,1.631898,-2.743061,-0.508318,0.466940,-0.539384,0.549831,...,-0.091786,0.658891,-0.031013,-0.731506,0.524027,0.125519,0.365563,-0.560976,0.263906,0.508082
4,14.283263,-4.295277,-6.995214,0.967853,-1.704958,4.325885,-2.462479,0.842244,-0.766773,-0.456736,...,-0.316532,0.544559,0.134363,-0.209656,0.609228,0.904788,0.271621,0.272284,0.548494,-0.602611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,9.753978,-1.717865,3.026078,-1.185909,1.547944,-1.744803,0.196555,0.326706,0.256284,-0.103832,...,0.369623,-0.025368,-0.089112,-0.140927,0.237865,0.104186,-0.045017,-0.315925,0.342902,0.379778
7456,13.232554,-1.515317,4.225104,-1.335346,-2.424473,-0.447255,-1.318402,0.501551,-0.148292,1.943560,...,-0.428183,-0.691371,0.047509,0.580997,-0.004254,0.392903,-0.198343,0.210394,-0.106948,0.222788
7457,12.637814,3.273628,-0.260518,-1.326048,1.966923,-3.236486,-0.291114,-0.500357,-2.008441,-2.200970,...,-0.925530,-0.854843,0.811911,0.294073,-0.262966,-0.409887,-0.882525,0.514422,0.684695,0.056088
7458,5.612700,0.939368,-0.881728,1.390165,-0.815870,0.185313,-1.668138,0.891430,0.132829,0.533787,...,0.288163,-0.013864,0.117686,-0.157926,0.268565,-0.252333,-0.186039,-0.290936,0.396492,0.035808


In [22]:
df_merge_coauthors_sentences.fillna(0,inplace =True)
df_coauthors_features.fillna(0,inplace =True)

In [41]:
X_train,X_test,y_train,y_test = train_test_split(df_merge_coauthors_sentences,ohe_prolific_labels,test_size=0.1,random_state=90051)

## classifierchain model (无深度学习目前最好的model) Logistic Regression

In [54]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
chain = ClassifierChain(base_lr, order='random', random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
chain.fit(X_scaled_train, y_train)
predictions = chain.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

array([0.75675676, 0.        , 0.36363636, 0.        , 0.375     ,
       0.375     , 0.28571429, 0.73684211, 0.        , 0.15384615,
       0.16      , 0.625     , 0.38095238, 0.5       , 0.44444444,
       0.6       , 0.51851852, 0.42857143, 0.        , 0.66666667,
       0.        , 0.75      , 0.        , 0.15384615, 0.13333333,
       0.33333333, 0.51428571, 0.11111111, 0.26666667, 0.4       ,
       0.30769231, 0.6       , 0.57142857, 0.85714286, 0.15384615,
       0.125     , 0.        , 0.54545455, 0.33333333, 0.45454545,
       0.88888889, 0.5       , 0.5       , 0.6       , 0.57142857,
       0.8       , 0.53333333, 0.72727273, 0.        , 0.33333333,
       0.22222222, 0.28571429, 0.        , 0.45714286, 0.33333333,
       0.72727273, 0.36363636, 0.5       , 0.33333333, 0.5       ,
       0.5       , 0.53333333, 0.15384615, 0.625     , 0.36363636,
       0.56      , 0.54545455, 0.25      , 0.28571429, 0.28571429,
       0.5       , 0.30769231, 0.35294118, 0.16666667, 0.34782

In [55]:
f1_score(y_test, predictions, average='samples')

0.38054884743892786

## MLP

In [52]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test= scaler.transform(X_test)

clf = MLPClassifier(random_state=0, max_iter=300, verbose = True,hidden_layer_sizes=(256,128),early_stopping = True)

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
f1_score(y_test, predictions, average=None)

Iteration 1, loss = 31.79902727
Validation score: 0.000000
Iteration 2, loss = 7.81017444
Validation score: 0.000000
Iteration 3, loss = 6.63391503
Validation score: 0.000000
Iteration 4, loss = 6.02741855
Validation score: 0.008929
Iteration 5, loss = 5.49953782
Validation score: 0.044643
Iteration 6, loss = 5.02517867
Validation score: 0.072917
Iteration 7, loss = 4.60345319
Validation score: 0.110119
Iteration 8, loss = 4.22720906
Validation score: 0.125000
Iteration 9, loss = 3.88993986
Validation score: 0.130952
Iteration 10, loss = 3.58184843
Validation score: 0.141369
Iteration 11, loss = 3.31047897
Validation score: 0.159226
Iteration 12, loss = 3.06293644
Validation score: 0.181548
Iteration 13, loss = 2.84272363
Validation score: 0.190476
Iteration 14, loss = 2.63672073
Validation score: 0.199405
Iteration 15, loss = 2.45939155
Validation score: 0.217262
Iteration 16, loss = 2.29497101
Validation score: 0.227679
Iteration 17, loss = 2.13432574
Validation score: 0.236607
Itera

array([0.66666667, 0.5       , 0.4       , 0.        , 0.2       ,
       0.2       , 0.51851852, 0.75      , 0.        , 0.22222222,
       0.4137931 , 0.30769231, 0.47619048, 0.69565217, 0.54545455,
       0.61538462, 0.51851852, 0.30769231, 0.5       , 0.85714286,
       0.        , 0.75      , 0.28571429, 0.625     , 0.125     ,
       0.35294118, 0.51428571, 0.22222222, 0.16666667, 0.36363636,
       0.4       , 0.63157895, 0.68571429, 0.66666667, 0.25      ,
       0.28571429, 0.        , 0.45454545, 0.42105263, 0.15384615,
       0.94736842, 0.35294118, 0.4       , 0.28571429, 0.57142857,
       0.66666667, 0.30769231, 0.58064516, 0.        , 0.28571429,
       0.36363636, 0.33333333, 0.36363636, 0.4       , 0.4       ,
       0.88888889, 0.44444444, 0.53333333, 0.18181818, 0.63157895,
       0.        , 0.57142857, 0.28571429, 0.83333333, 0.36363636,
       0.5       , 0.2       , 0.33333333, 0.5       , 0.28571429,
       0.375     , 0.25      , 0.46153846, 0.        , 0.38095

In [53]:
f1_score(y_test, predictions, average='samples')

0.3716136856887527

In [50]:
coeffs = clf.coefs_
print(coeffs[-1])

[-0.01151178 -0.16599307 -0.11961255  0.12535166  0.13238064 -0.06405664
  0.06667251 -0.06386235 -0.14440664 -0.0981975  -0.04007842  0.13363778
  0.11128144 -0.01364952 -0.0478771  -0.10713261  0.08398104 -0.14768459
  0.00147074  0.09296331  0.0522294   0.11306657 -0.01941741  0.05866192
  0.01667271  0.12322454  0.04013111 -0.09860826 -0.15367201  0.09906988
 -0.10009514 -0.00134247  0.1114722  -0.15391926 -0.13113556 -0.02518498
 -0.14893271  0.08648506  0.08131968  0.12376591 -0.04942399 -0.05699672
  0.04130821 -0.14802018 -0.07386962  0.0274661   0.04767594  0.13270523
 -0.00933826  0.05888962 -0.08631633  0.07933198 -0.05050917  0.02515419
 -0.09911816 -0.08107735  0.09662001  0.00372044  0.07772584 -0.0094909
  0.12734714  0.03005669  0.00475566 -0.11592743 -0.06710797 -0.06565336
 -0.02643577 -0.16209986  0.02639124  0.0605552   0.04209808  0.0880524
  0.00749032 -0.06159157 -0.14098471 -0.01001051  0.11116902 -0.16600983
 -0.00752348 -0.01272066  0.02803784 -0.05469573  0.0

## MultiOutputClassifier

In [ ]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
clf = MultiOutputClassifier(base_lr).fit(X_scaled_train, y_train)
predictions = clf.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

array([0.78947368, 0.5       , 0.28571429, 0.22222222, 0.61538462,
       0.28571429, 0.52173913, 0.70588235, 0.33333333, 0.4       ,
       0.375     , 0.58823529, 0.1875    , 0.4       , 0.88888889,
       0.54545455, 0.4       , 0.46153846, 0.4       , 0.66666667,
       0.        , 0.5       , 0.36363636, 0.375     , 0.        ,
       0.43478261, 0.47058824, 0.38095238, 0.26666667, 0.625     ,
       0.        , 0.4       , 0.62068966, 0.8       , 0.44444444,
       0.5       , 0.        , 0.45454545, 0.54545455, 0.23529412,
       0.53333333, 0.33333333, 0.4       , 0.85714286, 0.6       ,
       0.71428571, 0.22222222, 0.70967742, 0.        , 0.5       ,
       0.28571429, 0.28571429, 0.22222222, 0.36842105, 0.        ,
       0.90909091, 0.42857143, 0.73684211, 0.53333333, 0.63157895,
       0.66666667, 0.70588235, 0.2       , 0.66666667, 0.33333333,
       0.56      , 0.57142857, 0.4       , 0.28571429, 0.30769231,
       0.66666667, 0.33333333, 0.52631579, 0.22222222, 0.38461

In [ ]:
f1_score(y_test, predictions, average='samples')

0.40389378271415804

## Test set

In [ ]:
#Test
# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

test_string_sentences = convert_to_string(df_test['sentences'])
test_sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
test_sentences_counts = test_sentences_vectorizer.fit_transform(test_string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

test_string_coauthor = convert_to_string(df_test['coauthors'])
test_coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
test_coauthor_counts = coauthor_vectorizer.fit_transform(test_string_coauthor).toarray()



test_coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_test_coauthors_features = pd.DataFrame(test_coauthor_counts)
df_test_coauthors_features = df_test_coauthors_features.astype('float')
df_test_coauthors_features.columns = test_coauthors_features_name

test_sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_test_sentence_features = pd.DataFrame(test_sentences_counts)
df_test_sentence_features = df_test_sentence_features.astype('float')
df_test_sentence_features.columns = test_sentence_features_name


df_test_merge_coauthors_sentences = pd.concat([df_test_coauthors_features, df_test_sentence_features], axis = 1)




In [ ]:
df_test_merge_coauthors_sentences

In [ ]:
X_scaled_predict= scaler.transform(df_test_merge_coauthors_sentences)

In [ ]:
predition = chain.predict(X_scaled_predict)

In [ ]:
predition

In [ ]:
predicted_list = [i.tolist() for i in predition]
total_index_list = []
for i in predicted_list:
  index_list = []
  for j,k in enumerate(i):
    if k == 1:
      index_list.append(str(j))
  if len(index_list) != 0:
    total_index_list.append(index_list)
  else:
    total_index_list.append([str(-1)])

predicted_str = []
for i in total_index_list:
  a = ','.join(i)
  predicted_str.append(a)

In [ ]:

df_output = pd.DataFrame({'ID': list(range(800)),'Predict':predicted_str})
df_output.to_csv('predicted_result.csv',index = False)